# A neural network to classify images of clothing

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np

We're using a dataset called 'Fashion MNIST'. GitHub: https://github.com/zalandoresearch/fashion-mnist.

This dataset contains 60,000 training images and 10,000 testing images. Each image is a 28x28px grayscale image of a fashion item, associated with a label from 10 classes. The task is to predict the class of the image (e.g. t-shirt , dress, etc.). Each image has a resolution of 28x28 pixels and is grayscale (1 channel). The grayscale values range from 0 to 255, where 0 represents black and 255 represents white.

| Label | Item |
|-------|------|
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |
<p></p>

<div style="display: flex; justify-content: center;">
    <div style="margin-right: 20px;">
        <img src="mnist.png" width=400px alt="mnist dataset image">
    </div>
    <div>
        <img src="mnist.gif" width=600px alt="mnist classes">
    </div>
</div>

tfds.load loads

`split` specifies which splits of the dataset is to be loaded. You set as_supervised to True to ensure that the loaded tf.data.Dataset will have a 2-tuple structure (input, label).

`ds_train` and `ds_test` are of type `tf.data.Dataset`. `ds_train` has 60,000 images which will be used for training the model. `ds_test` has 10,000 images which will be used for evaluating the model.

In [20]:
# Define, load and configure data
(ds_train, ds_test), info = tfds.load('fashion_mnist', split=['train', 'test'], with_info=True, as_supervised=True)

In [21]:
header = info.features['label'].num_classes
print(f"Number of classes: {header}")
print(f"Number of training examples: {info.splits['train'].num_examples}")
print(f"Number of test examples: {info.splits['test'].num_examples}")

Number of classes: 10
Number of training examples: 60000
Number of test examples: 10000


In [29]:
# Minium and maximum values before normalization
image_batch, labels_batch = next(iter(ds_train))
print("Before normalization ->", np.min(image_batch[0]), np.max(image_batch[0]))

Before normalization -> 0 227


> `Batch size` refers to the number of training examples utilized in one iteration.

We'll be normalizing the data to exist between 0 and 1.

code given below uses the `map()` function of `tf.data.Dataset` to apply the normalization to images in `ds_train` and `ds_test`. Since the pixel values are of type `tf.uint8`, the `tf.cast` function is used to convert them to `tf.float32` and then divide by 255.0. The dataset is also converted into batches by calling the `batch()` method with BATCH_SIZE as the argument.

In [30]:
# Define batch size
BATCH_SIZE = 32

In [31]:
# Normalize and batch process the dataset
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).batch(BATCH_SIZE)
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).batch(BATCH_SIZE)

In [32]:
# Min and max values of the batch after normalization
image_batch, labels_batch = next(iter(ds_train))
print("After normalization ->", np.min(image_batch[0]), np.max(image_batch[0]))

After normalization -> 0.0 1.0


## Design, Compile and Train